# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import holidays
from pathlib import Path

%matplotlib inline

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_selector as selector
from sklearn.feature_extraction import DictVectorizer

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV


from xgboost import XGBRegressor
import xgboost as xgb

from lightgbm import LGBMRegressor
import lightgbm as lgb

# CSV Reads

In [ ]:
data_airports = pd.read_csv(
    "/home/antoine/projects/forecasting/data/AODB_airport_master.csv"
)

data_countries = pd.read_csv(
    "/home/antoine/projects/forecasting/data/AODB_country_master.csv"
)

all_files = list(
    Path("/home/antoine/projects/forecasting/data/planned flights").glob("*.csv")
)
data_planned = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

all_files = list(
    Path("/home/antoine/projects/forecasting/data/KIX_AODB_data").glob("*.csv")
)
data_actual = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
# filter out 17 January, it's today
mask = pd.to_datetime(data_actual["[Arr] SIBT"]).dt.floor("d") != pd.to_datetime(
    "2023-1-17"
).floor("d")
data_actual = data_actual[mask].copy()

# Data cleaning & augmenting

In [ ]:
def data_cleaning(data_actual, training: bool = True):
    # generate proper format
    mask = data_actual["[Dep] Flight Designator"].notna()
    data_dep = pd.DataFrame.from_dict(
        {
            "L Board Pax": pd.to_numeric(
                data_actual[mask]["[Dep]  L Board Pax"], errors="coerce"
            ).to_list(),
            "Routing-FirstLeg": data_actual[mask]["[Dep] Routing"]
            .apply(lambda x: x[-4:])
            .to_list(),
            "Datetime": pd.to_datetime(data_actual[mask]["[Dep] SOBT"]).to_list(),
            "Service Type": data_actual[mask]["[Dep] Service Type"].to_list(),
            "Traffic Type": data_actual[mask]["[Dep] Traffic Type"].to_list(),
            "UniqueID": pd.to_datetime(data_actual[mask]["[Dep] SOBT"]).dt.strftime(
                "%Y/%m/%d"
            )
            + " "
            + data_actual[mask]["[Dep] Flight Designator"],
            "Capacity": pd.to_numeric(
                data_actual[mask]["[Dep] Capacity"], errors="coerce"
            ).to_list(),
            "Flight Designator": data_actual[mask]["[Dep] Flight Designator"],
        }
    )
    data_dep["Direction"] = "departure"

    mask = data_actual["[Arr] Flight Designator"].notna()
    data_arr = pd.DataFrame.from_dict(
        {
            "L Board Pax": pd.to_numeric(
                data_actual[mask]["[Arr]  L Board Pax"], errors="coerce"
            ).to_list(),
            "Routing-FirstLeg": data_actual[mask]["[Arr] Routing"]
            .apply(lambda x: x[0:4])
            .to_list(),
            "Datetime": pd.to_datetime(data_actual[mask]["[Arr] SIBT"]).to_list(),
            "Service Type": data_actual[mask]["[Arr] Service Type"].to_list(),
            "Traffic Type": data_actual[mask]["[Arr] Traffic Type"].to_list(),
            "UniqueID": pd.to_datetime(data_actual[mask]["[Arr] SIBT"]).dt.strftime(
                "%Y/%m/%d"
            )
            + " "
            + data_actual[mask]["[Arr] Flight Designator"],
            "Capacity": pd.to_numeric(
                data_actual[mask]["[Arr] Capacity"], errors="coerce"
            ).to_list(),
            "Flight Designator": data_actual[mask]["[Arr] Flight Designator"],
        }
    )
    data_arr["Direction"] = "arrival"

    data_concat = pd.concat([data_dep, data_arr]).reset_index(drop=True)
    data_concat.drop_duplicates(subset=["UniqueID"], inplace=True)
    data = data_concat.copy()

    # select only useful columns
    data = data[
        [
            "Service Type",  # string
            "Traffic Type",  # string
            "Capacity",  # to convert to int
            "L Board Pax",  # int already
            "Direction",  # string
            "Datetime",  # date to convert to int for year/month/date/hour
            "Routing-FirstLeg",  # string, should be country
            "UniqueID",
            "Flight Designator",
        ]
    ].copy()

    # filter out rows with irrelevant values
    mask = data["Service Type"].isin(["C", "G", "J"])
    data = data[mask].copy()

    # change capacity to numerical
    data["Capacity"] = pd.to_numeric(data["Capacity"], errors="coerce")
    data["L Board Pax"] = pd.to_numeric(data["L Board Pax"], errors="coerce")

    # split date into year month day
    data["Datetime"] = pd.to_datetime(data["Datetime"])
    data["Year"] = data["Datetime"].apply(lambda x: x.year)
    data["Month"] = data["Datetime"].apply(lambda x: x.month)
    data["Day"] = data["Datetime"].apply(lambda x: x.day)
    data["Hour"] = data["Datetime"].apply(lambda x: x.hour)
    data["LinearDate"] = (
        pd.to_datetime(data["Datetime"]) - pd.to_datetime("2022-06-01")
    ) / np.timedelta64(1, "D")

    # for training, take relevant flights and calculate LF
    if training:
        mask = data["L Board Pax"] > 10
        data = data[mask].copy()
        data["L Board Pax"] = data["L Board Pax"].astype("int")
        data["Load Factor"] = data["L Board Pax"] / data["Capacity"]
        # remove wrong and na values
        data = data[
            (data["Load Factor"] < 1)
            & (data["Load Factor"] > 0)
            & (data["Load Factor"].notna())
        ]
    else:
        data["L Board Pax"] = np.nan
        data["Load Factor"] = np.nan

    # change routing to Country name then to Country code
    repl = (
        data_airports[["ICAO", "Country"]].set_index("ICAO").T.to_dict(orient="records")
    )
    data["Country"] = data["Routing-FirstLeg"].map(*repl)
    repl_country = (
        data_countries[["Name", "ISO-3166-1 alpha-2"]]
        .set_index("Name")
        .T.to_dict(orient="records")
    )
    data["Country"] = data["Country"].map(*repl_country)

    # holidays
    data["HolidayJP"] = 0
    data["HolidayOrigin"] = 0

    dct_holiday = {
        country_code: holidays.country_holidays(country_code)
        for country_code in data["Country"].unique()
        if hasattr(holidays, country_code)
    }

    for index, row in data.iterrows():
        # domestic holiday
        if row["Datetime"] in dct_holiday["JP"]:
            data.loc[index, "HolidayJP"] = 1
        # overseas holiday
        if row["Country"] in dct_holiday.keys():
            if row["Datetime"] in dct_holiday[row["Country"]]:
                data.loc[index, "HolidayOrigin"] = 1

    # drop old columns
    data.drop(
        [
            # "L Board Pax",
            # "Capacity",
            "Routing-FirstLeg",
            # "Datetime",
        ],
        axis="columns",
        inplace=True,
    )

    # change types for categories
    data["Service Type"] = data["Service Type"].astype("category")
    data["Traffic Type"] = data["Traffic Type"].astype("category")
    data["Direction"] = data["Direction"].astype("category")
    data["Country"] = data["Country"].astype("category")

    data["Month"] = data["Month"].astype("category")
    data["Day"] = data["Day"].astype("category")
    data["Hour"] = data["Hour"].astype("category")

    return data

In [ ]:
# Determine capacity for future flights
# step 1: create a table of flight and capacity from actual data
master_capacity = (
    data_cleaning(data_actual)
    .sort_values(by="Datetime", ascending=False)[["Flight Designator", "Capacity"]]
    .groupby("Flight Designator")
    .tail(1)
)
# when no info, set mean
master_capacity["Capacity"].fillna(master_capacity["Capacity"].mean(), inplace=True)

# step 2 assign to future flight based on flight designator
data_forecast_clean = (
    data_cleaning(data_planned, training=False)
    .drop("Capacity", axis=1)
    .join(master_capacity.set_index("Flight Designator"), on="Flight Designator")
)
# when no info, set mean
data_forecast_clean["Capacity"].fillna(
    data_forecast_clean["Capacity"].mean(), inplace=True
)

# prepare training data too
data_actual_clean = data_cleaning(data_actual)

# Pipeline creation and first fit

In [ ]:
# pipeline creation
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)
categorical_transformer = OneHotEncoder(handle_unknown="ignore")
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, selector(dtype_exclude="category")),
        ("cat", categorical_transformer, selector(dtype_include="category")),
    ]
)
model = XGBRegressor()
# model = LGBMRegressor()
regressor = Pipeline(steps=[("preprocessor", preprocessor), ("regressor", model)])

In [ ]:
# split dataset for X, y and train, test
X = data_actual_clean.drop(
    [
        "Capacity",
        "L Board Pax",
        "Datetime",
        "UniqueID",
        "Flight Designator",
        "Load Factor",
    ],
    axis=1,
)
y = data_actual_clean["Load Factor"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
# training model
regressor.fit(X_train, y_train)
print("training score: %.3f" % regressor.score(X_train, y_train))
print("model score: %.3f" % regressor.score(X_test, y_test))

In [ ]:
regressor["regressor"].get_booster().feature_names = list(
    regressor["preprocessor"].get_feature_names_out()
)
xgb.plot_importance(regressor["regressor"], ax=plt.gca(), max_num_features=10)

# Test model on future schedule

In [ ]:
# split dataset for X, y and train, test
X_forecast = data_forecast_clean.drop(
    [
        "Capacity",
        "L Board Pax",
        "Datetime",
        "UniqueID",
        "Flight Designator",
        "Load Factor",
    ],
    axis=1,
)
data_forecast_clean["Load Factor"] = regressor.predict(X_forecast)
data_forecast_clean["L Board Pax"] = (
    data_forecast_clean["Capacity"] * data_forecast_clean["Load Factor"]
)

In [ ]:
# prepare graphs for Pax count international departure

# actual
data_actual_result = data_actual_clean.set_index("Datetime", drop=True)
mask = (data_actual_result["Traffic Type"] == "INTERNATIONAL") & (
    data_actual_result["Direction"] == "departure"
)
plot_actual_pax = data_actual_result[mask]["L Board Pax"].groupby(
    pd.to_datetime(data_actual_result[mask].index).date
)

plot_actual_LF = data_actual_result[mask]["Load Factor"].groupby(
    pd.to_datetime(data_actual_result[mask].index).date
)

# forecast
data_forecast_result = data_forecast_clean.set_index("Datetime", drop=True)
mask = (data_forecast_result["Traffic Type"] == "INTERNATIONAL") & (
    data_forecast_result["Direction"] == "departure"
)
plot_forecast_pax = data_forecast_result[mask]["L Board Pax"].groupby(
    pd.to_datetime(data_forecast_result[mask].index).date
)
plot_forecast_LF = data_forecast_result[mask]["Load Factor"].groupby(
    pd.to_datetime(data_forecast_result[mask].index).date
)

In [ ]:
fig, ax = plt.subplots(
    ncols=3,
    figsize=(16, 6),
)
ax[0].plot(plot_forecast_pax.agg("count"))
ax[0].plot(plot_actual_pax.agg("count"))
ax[0].set(title="Flight number")
ax[1].plot(plot_forecast_pax.agg("sum"))
ax[1].plot(plot_actual_pax.agg("sum"))
ax[1].set(title="Pax number")
ax[2].plot(plot_forecast_LF.agg("mean"))
ax[2].plot(plot_actual_LF.agg("mean"))
ax[2].set(title="Load Factor")

In [ ]:
out_path = Path("/home/antoine/projects/forecasting/output") / "forecast.csv"
data_forecast_result[["UniqueID", "L Board Pax"]].reset_index().to_csv(out_path)